## Технология меторекомендаций

In [39]:
from surprise import KNNWithMeans, KNNBasic, SVDpp
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split, cross_validate

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

from sklearn.ensemble import RandomForestClassifier


import pandas as pd
#import numpy as np

### Загружаем набор данных и откладываем данные для теста

In [2]:
data = Dataset.load_builtin('ml-100k')
trainset, testset = train_test_split(data, test_size=.15)

### Обучаем первый алгоритм

In [3]:



# первый алгоритм
algo = SVDpp()
    
algo.fit(trainset)
predictions = algo.test(testset)

accuracy.rmse(predictions)

RMSE: 0.9200


0.9199970722879779

### Обучаем второй алгоритм

In [4]:
# второй алгоритм
algo2 = KNNBasic()
algo2.fit(trainset)
predictions2 = algo2.test(testset)

accuracy.rmse(predictions2)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9777


0.9776612479627784

### Обучаем третий алгоритм

In [79]:
# третий алгоритм
algo3 = KNNWithMeans()
algo3.fit(trainset)
predictions3 = algo3.test(testset)

accuracy.rmse(predictions3)


Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9451


0.9451459604344402

### На трейне делаем предсказание каждым из трех алгоритмов и отбираем лучший для каждого предсказания

In [49]:
inc = 0
trainset_for_meta = []
train_y = []
for row in trainset.all_ratings():
    inc += 1
    #print('row={}'.format(row))
    uid,iid,sc0 = str(row[0]),str(row[1]),row[2]

    #print('uid={},iid={},sc0={}'.format(uid,iid,sc0))
    pred1 = algo.predict(uid, iid,verbose=False)
    pred2 = algo2.predict(uid, iid,verbose=False)
    pred3 = algo3.predict(uid, iid,verbose=False)

    #print('pred1: {} ; pred2: {}; pred3: {}'.format(pred1.est,pred2.est,pred3.est))

    dt = {1:abs(sc0-pred1.est), 
          2:abs(sc0-pred2.est), 
          3:abs(sc0-pred3.est)}

    #print('sc0: {} ; dt: {}'.format(sc0,dt))
    key_min = min(dt.keys(), key=(lambda k: dt[k]))
    #print('key: {} ; Best score: {}'.format(str(key_min),dt[key_min]))
    #print('new row: {}'.format((uid, iid, key_min)))
    trainset_for_meta.append((uid, iid))
    train_y.append(key_min)
    #if inc > 1000:
    #    break
        


### Подготавливаетм тестовые данные для алгоритма классификации из библиотеки sklearn

In [80]:
inc = 0
testset_for_meta = []
testset_y = []
for row in testset:
    inc += 1
    #print('row={}'.format(row))
    uid,iid,sc0 = str(row[0]),str(row[1]),row[2]
    testset_for_meta.append((uid, iid))
    testset_y.append(sc0)

    #if inc > 5:
    #    break



### На обучаемом наборе данных с метками лучшего алгоритма обучаетм модель классификации. Затем делаем предсказание на тесте какой алгоритм будет лучшим для каждого объекта из теста

In [83]:
from sklearn.metrics import average_precision_score

model_1 = RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0)
model_1.fit(trainset_for_meta, train_y)
test_pred = model_1.predict(testset_for_meta)


### По предсказанному лучшему алгоритму делаем предсказание тем алгоритмом который предсказан

In [92]:
inc = 0
testset_pred = []
for row in test_pred:
    test_row = testset[inc]
    #print('test_row ={}'.format(test_row))
    uid,iid,sc0 = test_row[0],test_row[1],test_row[2]
    if row==1:
        pred = algo.predict(uid, iid,sc0,verbose=False)
        #print('Algo={}'.format(pred1.est))
    elif row==2:
        pred = algo2.predict(uid, iid,sc0,verbose=False)
        #print('Algo={}'.format(pred2.est))
    elif row==3:
        pred = algo3.predict(uid, iid,sc0,verbose=False)
        #print('Algo={}'.format(pred3.est))
    else: 
        print('Not switch={}'.format(row))

    testset_pred.append(pred)

    #inc += 1
    #if inc > 5:
    #    break




### Проверка показале, что фокус не удался!
### Однако цель занятия достигнута - закреплен понравивщееся технология на пректике.

In [93]:
accuracy.rmse(testset_pred)

RMSE: 1.4181


1.4181038197903417